In [ ]:
max_match_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_match_ts", debugValue="2026-01-01 00:00:00")
max_delivery_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_delivery_ts", debugValue="2026-01-01 00:00:00")

print(max_match_ts, max_delivery_ts)

In [ ]:
df = spark.sql(f"SELECT * FROM bronze.delivery_loading WHERE ingestion_ts <= '{max_delivery_ts}'")

In [ ]:
players_df = df.select(["batsman","non_striker","fielder"])

In [ ]:
from pyspark.sql.functions import col, lit, current_timestamp

# .distinct() ensures we don't try to insert the same umpire twice from the same batch
source_player_df = players_df.select(col("batsman").alias("player_name")) \
    .union(players_df.select(col("non_striker").alias("player_name"))) \
    .union(players_df.select(col("fielder").alias("player_name"))) \
    .filter(col("player_name").isNotNull()) \
    .distinct()

In [ ]:
from pyspark.sql.functions import col, lit, current_timestamp, year

final_player_df = source_player_df.select(
    col("player_name"),
    lit(None).cast("date").alias("date_of_birth"),
    lit(None).cast("string").alias("country"),
    lit(None).cast("string").alias("role"),
    lit(None).cast("boolean").alias("is_wicketkeeper"),
    lit(None).cast("string").alias("batting_hand"),
    lit(None).cast("string").alias("bowling_style"),
    lit(None).cast("int").alias("retirement_year"),
    lit(None).cast("int").alias("ipl_debut_year"),
    current_timestamp().alias("created_at")
)


In [ ]:
final_player_df.show(5)

In [ ]:
# 3. Create Temp View for SQL
final_player_df.createOrReplaceTempView("source_new_player")

In [ ]:
# 4. Perform MERGE (Upsert) into dim_player
spark.sql("""
MERGE INTO silver.dim_player AS target
USING source_new_player AS source
ON target.player_name = source.player_name

WHEN NOT MATCHED THEN
  INSERT (
    player_name,
    date_of_birth,
    country,
    role,
    is_wicketkeeper,
    batting_hand,
    bowling_style,
    retirement_year,
    ipl_debut_year,
    created_at
  )
  VALUES (
    source.player_name,
    source.date_of_birth,
    source.country,
    source.role,
    source.is_wicketkeeper,
    source.batting_hand,
    source.bowling_style,
    source.retirement_year,
    source.ipl_debut_year,
    source.created_at
  )
""")


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]